---
title: "LS_BS_4th_6조 - Ames Housing 화재 리스크 분석"
format:
  dashboard:
    theme: morph
    logo: images/beetle.png
    sidebar: true
    page-navigation: true
    reader-mode: false
    nav-buttons:
      - icon: github
        href: https://github.com/your-repo-link
      - icon: globe
        href: https://your-site-link
---


## 📄 2025.4.22 회의록 – 주택 화재 리스크 및 보험금 예측 모델링

**주제:** 심슨 가족이 Ames에 집을 샀을 때, 화재 발생 시 보험금은 얼마일까?

---

## 🔹 0. 인트로 – 랜덤 입주 & 문제 제기


In [ ]:
import pandas as pd
import numpy as np
np.random.seed(2025)
dataloader = DataLoader()
dataset = dataloader.load_data()
df = dataset
simpsons_home = df.sample(1)
simpsons_home[["Neighborhood", "YearBuilt", "SalePrice"]]

---

## 🔹 1. 화재 위험 지역 시각화 {.tabset}

### 🔧 화재 위험 점수 생성

In [ ]:
def assign_risk(exterior):
    if exterior in ['Wd Sdng', 'WdShing', 'Plywood']: return 'High'
    elif exterior in ['VinylSd', 'MetalSd']: return 'Medium'
    return 'Low'

df['FireRiskGrade'] = df['Exterior1st'].apply(assign_risk)
df['FireRiskScore'] = df['FireRiskGrade'].map({'Low': 1, 'Medium': 2, 'High': 3})

### 🧪 지역별 위험도 통계 검정

In [ ]:
from scipy.stats import ttest_ind
df_low = df[df.FireRiskGrade == 'Low']
df_high = df[df.FireRiskGrade == 'High']
ttest_ind(df_low['SalePrice'], df_high['SalePrice'])

### 🗺️ 지도 시각화 (Plotly 예시)

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude",
                         color="FireRiskGrade", mapbox_style="open-street-map",
                         zoom=11, title="화재 위험 등급별 위치 시각화")
fig.show()

---

## 🔹 2. 화재 대응 자원 분포 분석


In [ ]:
df['ResourceScore'] = (
    (df['PoolArea'] > 0).astype(int) +
    (df['Fireplaces'] > 0).astype(int) +
    (df['GarageCars'] > 0).astype(int) +
    (df['FullBath'] > 0).astype(int)
)

In [ ]:
fig = px.box(df, x="FireRiskGrade", y="ResourceScore",
             title="위험 등급별 자원 점수 Boxplot")
fig.show()

---

## 🔹 3. 피해액 예측 모델링

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = df[["FireRiskScore", "GrLivArea", "YearBuilt", "OverallQual"]]
y = df["SalePrice"] * 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
model = LinearRegression().fit(X_train, y_train)
df['ExpectedDamage'] = model.predict(X)

### ⚠ 모집단 대표성 검정

In [ ]:
pop_mean = 180000
sample_mean = df['SalePrice'].mean()
std = df['SalePrice'].std()
n = len(df)
z = (sample_mean - pop_mean) / (std / np.sqrt(n))
z

---

## 🔹 4. 예상 보험금 산정

In [ ]:
def insurance_calc(row):
    risk_weight = {'High': 1.2, 'Medium': 1.0, 'Low': 0.8}[row['FireRiskGrade']]
    resource_weight = 0.9 if row['ResourceScore'] >= 3 else 1.0
    aging_weight = 1.2 if (2025 - row['YearBuilt']) > 30 else 1.0
    return row['ExpectedDamage'] * risk_weight * resource_weight * aging_weight

df['ExpectedInsurance'] = df.apply(insurance_calc, axis=1)

In [ ]:
fig = px.histogram(df, x="ExpectedInsurance", nbins=30,
                   title="예상 보험금 분포")
fig.show()

---

## 🔹 5. 결론 및 보험금 산출 결과 정리 {.column width=100%}


In [ ]:
df['RiskScore'] = df['ExpectedDamage'] / df['ExpectedInsurance']
df.sort_values(by='ExpectedInsurance', ascending=False).head(5)[["Neighborhood", "SalePrice", "ExpectedDamage", "ExpectedInsurance"]]

::: {.card title="📌 분석 요약"}
- 심슨 가족의 주택을 포함한 화재 리스크 및 보험금 예측 결과 정리
- 예상 피해액과 주택 특성 기반 보험금 산출 완료
:::

> “우리가 예측한 보험금과 위험도를 기반으로,
> 보다 안전하고 합리적인 보험 정책 설계 및 주택 선택에 도움을 줄 수 있습니다.”